In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

# Параметры
LATENT_DIM = 50
BATCH_SIZE = 32
EPOCHS = 51  # Уменьшим для теста

# Загрузка данных
(x_train, _), (_, _) = keras.datasets.fashion_mnist.load_data()
x_train = x_train.astype('float32')
x_train = (x_train - 127.5) / 127.5
x_train = np.expand_dims(x_train, axis=-1)

# Убедимся, что у нас правильное количество примеров
num_samples = (x_train.shape[0] // BATCH_SIZE) * BATCH_SIZE
x_train = x_train[:num_samples]

dataset = tf.data.Dataset.from_tensor_slices(x_train)
dataset = dataset.shuffle(1000).batch(BATCH_SIZE)

#Архитектура модели
def create_models():
    # Генератор
    generator = keras.Sequential([
        layers.Dense(256, activation='relu', input_shape=(LATENT_DIM,)),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(1024, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(28*28*1, activation='tanh'),
        layers.Reshape((28, 28, 1))
    ])

    # Дискриминатор
    discriminator = keras.Sequential([
        layers.Flatten(input_shape=(28, 28, 1)),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])

    return generator, discriminator

generator, discriminator = create_models()

# params TRAIN
discriminator.compile(
    optimizer=keras.optimizers.Adam(0.0002, beta_1=0.5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

discriminator.trainable = False
gan_input = layers.Input(shape=(LATENT_DIM,))
gan_output = discriminator(generator(gan_input))
gan = keras.Model(gan_input, gan_output)
gan.compile(
    optimizer=keras.optimizers.Adam(0.0002, beta_1=0.5),
    loss='binary_crossentropy'
)

print(f"Генератор параметров: {generator.count_params():,}")
print(f"Дискриминатор параметров: {discriminator.count_params():,}")
print(f"Генератор обучаемых слоев: {len(generator.trainable_weights)}")
print(f"Дискриминатор обучаемых слоев: {len(discriminator.trainable_weights)}")
print(f"GAN обучаемых слоев: {len(gan.trainable_weights)}")

for epoch in range(EPOCHS):
    print("Эпоха", epoch)
    for i, real_images in enumerate(dataset):
        current_batch_size = real_images.shape[0]

        discriminator.trainable = True

        noise = tf.random.normal([current_batch_size, LATENT_DIM])
        fake_images = generator(noise, training=False)

        d_loss_real = discriminator.train_on_batch(
            real_images,
            tf.ones((current_batch_size, 1))
        )
        d_loss_fake = discriminator.train_on_batch(
            fake_images,
            tf.zeros((current_batch_size, 1))
        )
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        discriminator.trainable = False

        noise = tf.random.normal([current_batch_size, LATENT_DIM])
        g_loss = gan.train_on_batch(
            noise,
            tf.ones((current_batch_size, 1))
        )

    if epoch % 5 == 0:
        print(f'Epoch {epoch}, D Loss: {d_loss[0]:.4f}, D Acc: {d_loss[1]:.4f}, G Loss: {g_loss:.4f}')

        noise = tf.random.normal([16, LATENT_DIM])
        generated_images = generator(noise, training=False)

        fig, axes = plt.subplots(4, 4, figsize=(8, 8))
        for i, ax in enumerate(axes.flat):
            img = (generated_images[i, :, :, 0] + 1) / 2.0
            ax.imshow(img, cmap='gray')
            ax.axis('off')
        plt.savefig(f'ver2_fashion_epoch_{epoch}.png')
        plt.close()

print("Обучение завершено!")

Проверка моделей:
Генератор параметров: 1,480,720
Дискриминатор параметров: 533,505
Генератор обучаемых слоев: 14
Дискриминатор обучаемых слоев: 0
GAN обучаемых слоев: 14
Эпоха 0
Epoch 0, D Loss: 0.4495, D Acc: 0.7723, G Loss: 2.3386
Эпоха 1
Эпоха 2
Эпоха 3
Эпоха 4
Эпоха 5
Epoch 5, D Loss: 0.6267, D Acc: 0.6245, G Loss: 1.0716
Эпоха 6
Эпоха 7
Эпоха 8
Эпоха 9
Эпоха 10
Epoch 10, D Loss: 0.6528, D Acc: 0.5904, G Loss: 0.9212
Эпоха 11
Эпоха 12
Эпоха 13
Эпоха 14
Эпоха 15
Epoch 15, D Loss: 0.6644, D Acc: 0.5708, G Loss: 0.8581
Эпоха 16
Эпоха 17
Эпоха 18
Эпоха 19
Эпоха 20
Epoch 20, D Loss: 0.6707, D Acc: 0.5592, G Loss: 0.8236
Эпоха 21
Эпоха 22
Эпоха 23
Эпоха 24
Эпоха 25
Epoch 25, D Loss: 0.6742, D Acc: 0.5531, G Loss: 0.8020
Эпоха 26
Эпоха 27
Эпоха 28
Эпоха 29
Эпоха 30
Epoch 30, D Loss: 0.6764, D Acc: 0.5496, G Loss: 0.7875
Эпоха 31
Эпоха 32
Эпоха 33
Эпоха 34
Эпоха 35
Epoch 35, D Loss: 0.6774, D Acc: 0.5486, G Loss: 0.7776
Эпоха 36
Эпоха 37
Эпоха 38
Эпоха 39
Эпоха 40
Epoch 40, D Loss: 0.6777

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

# Параметры
LATENT_DIM = 50
BATCH_SIZE = 32
EPOCHS = 51

# Загрузка данных
(x_train, _), (_, _) = keras.datasets.fashion_mnist.load_data()
x_train = x_train.astype('float32')
x_train = (x_train - 127.5) / 127.5
x_train = np.expand_dims(x_train, axis=-1)

num_samples = (x_train.shape[0] // BATCH_SIZE) * BATCH_SIZE
x_train = x_train[:num_samples]

dataset = tf.data.Dataset.from_tensor_slices(x_train)
dataset = dataset.shuffle(1000).batch(BATCH_SIZE)

def create_models():
    generator = keras.Sequential([
        layers.Dense(256, activation='relu', input_shape=(LATENT_DIM,)),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(1024, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(28*28*1, activation='tanh'),
        layers.Reshape((28, 28, 1))
    ])

    discriminator = keras.Sequential([
        # Первый сверточный блок
        layers.Conv2D(64, (3,3), strides=(2,2), padding='same', input_shape=(28,28,1)),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),

        # Второй сверточный блок
        layers.Conv2D(128, (3,3), strides=(2,2), padding='same'),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),

        # Третий сверточный блок
        layers.Conv2D(256, (3,3), strides=(2,2), padding='same'),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),

        # Четвертый сверточный блок
        layers.Conv2D(512, (3,3), strides=(1,1), padding='same'),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),

        # Полносвязные слои
        layers.Flatten(),
        layers.Dense(1024, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])

    return generator, discriminator

generator, discriminator = create_models()

discriminator.compile(
    optimizer=keras.optimizers.Adam(0.0002, beta_1=0.5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

discriminator.trainable = False
gan_input = layers.Input(shape=(LATENT_DIM,))
gan_output = discriminator(generator(gan_input))
gan = keras.Model(gan_input, gan_output)
gan.compile(
    optimizer=keras.optimizers.Adam(0.0002, beta_1=0.5),
    loss='binary_crossentropy'
)

print(f"Генератор параметров: {generator.count_params():,}")
print(f"Дискриминатор параметров: {discriminator.count_params():,}")
print(f"Генератор обучаемых слоев: {len(generator.trainable_weights)}")
print(f"Дискриминатор обучаемых слоев: {len(discriminator.trainable_weights)}")
print(f"GAN обучаемых слоев: {len(gan.trainable_weights)}")

d_losses = []
g_losses = []
d_accs = []

for epoch in range(EPOCHS):
    print("Эпоха", epoch)
    epoch_d_losses = []
    epoch_g_losses = []
    epoch_d_accs = []

    for i, real_images in enumerate(dataset):
        current_batch_size = real_images.shape[0]

        discriminator.trainable = True

        noise = tf.random.normal([current_batch_size, LATENT_DIM])
        fake_images = generator(noise, training=False)

        d_loss_real = discriminator.train_on_batch(
            real_images,
            tf.ones((current_batch_size, 1))
        )
        d_loss_fake = discriminator.train_on_batch(
            fake_images,
            tf.zeros((current_batch_size, 1))
        )
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        discriminator.trainable = False

        noise = tf.random.normal([current_batch_size, LATENT_DIM])
        g_loss = gan.train_on_batch(
            noise,
            tf.ones((current_batch_size, 1))
        )

        epoch_d_losses.append(d_loss[0])
        epoch_g_losses.append(g_loss)
        epoch_d_accs.append(d_loss[1])

    avg_d_loss = np.mean(epoch_d_losses)
    avg_g_loss = np.mean(epoch_g_losses)
    avg_d_acc = np.mean(epoch_d_accs)

    d_losses.append(avg_d_loss)
    g_losses.append(avg_g_loss)
    d_accs.append(avg_d_acc)

    if epoch % 5 == 0:
        print(f'Epoch {epoch}, D Loss: {avg_d_loss:.4f}, D Acc: {avg_d_acc:.4f}, G Loss: {avg_g_loss:.4f}')

        noise = tf.random.normal([16, LATENT_DIM])
        generated_images = generator(noise, training=False)

        fig, axes = plt.subplots(4, 4, figsize=(8, 8))
        for i, ax in enumerate(axes.flat):
            img = (generated_images[i, :, :, 0] + 1) / 2.0
            ax.imshow(img, cmap='gray')
            ax.axis('off')
        plt.savefig(f'ver2_fashion_epoch_{epoch}.png')
        plt.close()

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(d_losses, label='D Loss')
plt.plot(g_losses, label='G Loss')
plt.legend()
plt.title('Losses')

plt.subplot(1, 2, 2)
plt.plot(d_accs, label='D Accuracy')
plt.legend()
plt.title('Discriminator Accuracy')
plt.savefig('training_metrics.png')
plt.close()

print("Обучение завершено!")

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Генератор параметров: 1,480,720
Дискриминатор параметров: 10,595,841
Генератор обучаемых слоев: 14
Дискриминатор обучаемых слоев: 0
GAN обучаемых слоев: 14
Эпоха 0
Epoch 0, D Loss: 0.1915, D Acc: 0.9248, G Loss: 4.7846
Эпоха 1
Эпоха 2
Эпоха 3
Эпоха 4
Эпоха 5
Epoch 5, D Loss: 0.1951, D Acc: 0.9277, G Loss: 3.1881
Эпоха 6
Эпоха 7
Эпоха 8
Эпоха 9
Эпоха 10
Epoch 10, D Loss: 0.1886, D Acc: 0.9314, G Loss: 3.0668
Эпоха 11
Эпоха 12
Эпоха 13
Эпоха 14
Эпоха 15
Epoch 15, D Loss: 0.1854, D Acc: 0.9331, G Loss: 3.0041
Эпоха 16
Эпоха 17
Эпоха 18
Эпоха 19
Эпоха 20
Epoch 20, D Loss: 0.1837, D Acc: 0.9340, G Loss: 2.9675
Эпоха 21
Эпоха 22
Эпоха 23
Эпоха 24
Эпоха 25
Epoch 25, D Loss: 0.1831, D Acc: 0.9343, G Loss: 2.9337
Эпоха 26
Эпоха 27
Эпоха 28
Эпоха 29
Эпоха 30
Epoch 30, D Loss: 0.1830, D Acc: 0.9344, G Loss: 2.9084
Эпоха 31
Эпоха 32
Эпоха 33
Эпоха 34
Эпоха 35
Epoch 35, D Loss: 0.1826, D Acc: 0.9346, G Loss: 2.8923
Эпоха 36
Эпоха 37
Эпоха 38
Эпоха 39
Эпоха 40
Epoch 40, D Loss: 0.1818, D Acc: 0.9349